## Lấy dữ liệu của từng product
1. Thêm thư viện

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import os, datetime, time, json

# define directories
IN_PATH = r".\data"
OUT_PATH = r".\documents"
LOG_SESSION = datetime.datetime.now().strftime("%d%m%Y%H%M")

In [ ]:
# get all file in data
FJoin = os.path.join
files = [FJoin(IN_PATH, f) for f in os.listdir(os.path.expanduser(IN_PATH))]

In [ ]:
# start webdriver Chrome
options = ChromeOptions()
options.add_argument("--headless=new")
web_driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=options)

In [ ]:
# create log
logs_file = open('logs/get_{0}.log'.format(LOG_SESSION),'w+',encoding='utf-8');

In [ ]:
# get products detail and save
logs_file.write("------------------------------------\nStart new session: {0}\n".format(LOG_SESSION))
count = 1000000
for f in files:
    with open(f, 'r', encoding='UTF-8') as file:
        lines = file.readlines()
        for l in lines:
            logs_file.write("Get product no {0} in {1}".format(lines.index(l)+1,f))
            print("Writing product {0}".format(count))
            web_driver.get(l)
            time.sleep(5)
            try:
                try:
                    document = {
                        "title" :  web_driver.find_element(By.XPATH, '//*[@id="productTitle"]').text,
                        "asin": web_driver.find_element(By.XPATH, '//*[@id="detailBullets_feature_div"]/ul/li[6]/span/span[2]').text,
                        "brand": web_driver.find_element(By.XPATH, '//*[@id="detailBullets_feature_div"]/ul/li[5]/span/span[2]').text,
                        "stars": web_driver.find_element(By.XPATH, '//*[@id="acrPopover"]/span[1]/a/span').text,
                        "reviewsCount": web_driver.find_element(By.XPATH, '//*[@id="acrCustomerReviewText"]').text,
                        "thumbnailImage": web_driver.find_element(By.XPATH, '//*[@id="landingImage"]').get_attribute("src"),
                        "breadCrumbs": [e.text for e in web_driver.find_elements(By.XPATH, '//*[@id="wayfinding-breadcrumbs_feature_div"]/ul/li')],
                        "description": web_driver.find_element(By.XPATH, '//*[@id="productDescription"]/p/span').text,
                        "price" : {
                            "value" :  web_driver.find_element(By.XPATH, '//*[@class="a-price-whole"]').text+'.'+web_driver.find_element(By.XPATH, '//*[@class="a-price-fraction"]').text,
                            "currency" : web_driver.find_element(By.XPATH, '//*[@class="a-price-symbol"]').text,
                        },
                        "url": l.replace("\n",""),
                    }
                except Exception as ex:
                    document = {
                        "title" :  web_driver.find_element(By.XPATH, '//*[@id="productTitle"]').text,
                        "asin": web_driver.find_element(By.XPATH, '//*[@id="productDetails_detailBullets_sections1"]/tbody/tr[1]/td').text,
                        "brand": web_driver.find_element(By.XPATH, '//*[@id="productDetails_techSpec_section_1"]/tbody/tr[1]/td').text,
                        "stars": web_driver.find_element(By.XPATH, '//*[@id="acrPopover"]/span[1]/a/span').text,
                        "reviewsCount": web_driver.find_element(By.XPATH, '//*[@id="acrCustomerReviewText"]').text,
                        "thumbnailImage": web_driver.find_element(By.XPATH, '//*[@id="landingImage"]').get_attribute("src"),
                        "breadCrumbs": [e.text for e in web_driver.find_elements(By.XPATH, '//*[@id="wayfinding-breadcrumbs_feature_div"]/ul/li')],
                        "description": web_driver.find_element(By.XPATH, '//*[@id="productDescription"]/p/span').text,
                        "price" : {
                            "value" :  web_driver.find_element(By.XPATH, '//*[@class="a-price-whole"]').text+'.'+web_driver.find_element(By.XPATH, '//*[@class="a-price-fraction"]').text,
                            "currency" : web_driver.find_element(By.XPATH, '//*[@class="a-price-symbol"]').text,
                        },
                        "url": l.replace("\n",""),
                    }
            except Exception as e:
                logs_file.write("Error in {0} at {1}".format(lines.index(l)+1,f))
                print("Error!!!")
            else:
                with open('documents/product{0}.json'.format(count), 'w+', encoding='utf-8') as file:
                    json.dump(document, file)
                print("Success!!!")
            count += 1

In [ ]:
logs_file.close()
web_driver.close()